## 🎨 Small Business Email Conversation Generator

This notebook demonstrates how to build a synthetic data generation pipeline for creating realistic email conversations between small business owners and their customers or professional contacts. The generated conversations can be used for training language models to better understand and handle small business communications.

In [1]:
from gretel_client.navigator import DataDesigner

### ✍️ Setting System Instruction
The system instructions guide our LLMs to generate authentic email conversations that match real-world business communications. These instructions ensure consistency in tone, style, and content while maintaining the natural variation you'd expect in small business emails.

In [2]:
system_instruction="\
    You are an expert at writing realistic emails. When given information about a person's job, \
    personality, and goals, write emails that sound exactly like they would write them. Match \
    their natural communication style - whether formal or casual, direct or diplomatic. \
    Include their common phrases and speech patterns. Consider their role, industry,\
    and relationship with the recipient. Make the tone and length appropriate for their objective. Avoid generic business language unless it matches how they actually write. Your emails should be indistinguishable from ones the person would write themselves."

### ⚙️ Data Designer Configuration
Instead of using a single configuration file, we build our pipeline step by step. This gives us fine-grained control over how we guide LLMs to produce authentic small business email conversations. The modular approach allows quick iteration and refinement of our data generation process.

In [3]:
## Create our DD Instance
data_designer = DataDesigner(
    model_suite='apache-2.0',
    special_system_instructions=system_instruction,
    endpoint="https://api.gretel.cloud",
    cache="yes"
)

[19:37:41] [INFO] 🦜 Using apache-2.0 model suite
Logged in as kirit.thadaka@gretel.ai ✅


### Use Structured Outputs to Definine the Email Structure

We use Pydantic to ensure our generated emails follow a consistent structure while maintaining flexibility in content. This helps maintain quality and reliability in our synthetic data.

In [4]:
from typing import Dict, List, Literal, Optional, Union, Any
from pydantic import BaseModel, Field, model_validator

class Email(BaseModel):
    """A single email in the email chain."""
    role: Literal["business_owner", "customer"] = Field(..., description="Which role is writing the message.")
    subject: str = Field(..., description="Email subject.")
    content: str = Field(..., description="Email contents.")


class EmailConversation(BaseModel):
    """An email conversation between a two people."""
    conversation: list[Email] = Field(..., description="List of all messages in the email chain.")

### 🌱 Categorical Seed Columns
Our seed columns define the context and characteristics of each email conversation. They include:

#### Business Context

* Business type and specific business category
* Business name generation
* Owner's communication style

#### Customer Context

* Customer personality and communication style
* Relationship with the business
* Previous interaction history

#### Conversation Parameters

* Primary goal of the email exchange
* Urgency level
* Overall tone
* Number of exchanges

Each seed category is carefully designed to create authentic variations in email conversations while maintaining realistic business scenarios.

In [5]:
data_designer.add_categorical_seed_column(
    name="business_type",
    description="The type of small business the owner operates",
    values=[
        "Food & Beverage",
        "Local Retail",
        "Skilled Trade",
        "Creative Services",
        "Health & Wellness"
    ],
    subcategories=[
        {
            "name": "specific_business",
            "values": {
                "Food & Beverage": [
                    "Family-run Bakery",
                    "Local Coffee Shop",
                    "Food Truck",
                    "Small Catering Business",
                    "Specialty Food Store"
                ],
                "Local Retail": [
                    "Independent Bookstore",
                    "Local Plant Nursery",
                    "Vintage Clothing Shop",
                    "Craft Supply Store",
                    "Local Pet Supply Store"
                ],
                "Skilled Trade": [
                    "Local Plumber",
                    "Independent Electrician",
                    "Small Carpentry Shop",
                    "Auto Repair Shop",
                    "Local Landscaping Service"
                ],
                "Creative Services": [
                    "Freelance Photographer",
                    "Independent Graphic Designer",
                    "Local Wedding Planner",
                    "Custom Jewelry Maker",
                    "Independent Interior Designer"
                ],
                "Health & Wellness": [
                    "Independent Massage Therapist",
                    "Local Yoga Studio",
                    "Family Chiropractor",
                    "Personal Training Business",
                    "Small Wellness Center"
                ]
            },
            "num_new_values_to_generate": 2
        }
    ]
)

In [6]:
data_designer.add_categorical_seed_column(
    name="owner_personality",
    description="The communication style and personality of the business owner",
    values=[
        "Professional and Formal",
        "Warm and Personal",
        "Direct and Efficient",
        "Creative and Casual",
        "Patient and Educational"
    ]
)

In [7]:
data_designer.add_categorical_seed_column(
    name="customer_personality",
    description="The personality type and communication style of the customer",
    values=[
        "Polite and Clear",
        "Frustrated but Respectful",
        "Demanding and Direct",
        "Detail-oriented",
        "Casual and Friendly",
        "Confused and Seeking Help"
    ]
)

In [8]:
data_designer.add_categorical_seed_column(
    name="conversation_goal",
    description="The primary objective of the email exchange in a small business context",
    values=[
        # Scheduling & Availability
        "Appointment Scheduling",     # For services like massage, training, repairs
        "Custom Order Discussion",    # For bakeries, jewelers, craftspeople
        "Consultation Request",       # For designers, wellness providers, trades
        
        # Service-Related
        "Quote Request",             # For trades, creative services, catering
        "Service Modification",      # Change to existing booking/order
        "Availability Check",        # Stock check, service times, special items
        
        # Customer Support
        "Progress Update Request",   # For ongoing projects, custom orders
        "Last-Minute Changes",       # Emergency scheduling, order modifications
        "Service Issue Resolution",  # Quality concerns, timing issues
        
        # Business Development
        "Collaboration Inquiry",     # Local business partnerships, events
        "Special Event Planning",    # Private bookings, workshops, classes
        "Custom Project Proposal",   # For creative services, skilled trades
        
        # Customer Experience
        "Follow-up Care",           # Post-service check-ins, maintenance tips
        "Detailed Instructions",     # Care instructions, product use guidance
        "Personal Recommendations",  # Product/service suggestions based on history
        
        # Administrative
        "Payment Arrangement",       # Deposits, payment plans, invoicing
        "Business Policy Question",  # Hours, booking policies, COVID protocols
        "Local Delivery Options"     # Delivery zones, timing, special handling
    ]
)

In [9]:
data_designer.add_categorical_seed_column(
    name="conversation_tone",
    description="The overall tone of the email exchange",
    values=[
        "Professional",
        "Friendly",
        "Formal",
        "Apologetic",
        "Appreciative",
        "Resolution-focused"
    ]
)

In [10]:
data_designer.add_categorical_seed_column(
    name="urgency_level",
    description="The urgency level and timing context of the small business email exchange",
    values=[
        # Same Day/Immediate
        "Emergency Service Needed",      # Urgent repairs, last-minute catering cancelation
        "Same-Day Modification",         # Changes to today's appointments/orders
        "Time-Critical Question",        # Questions about today's service/product
        
        # Near-Term
        "This Week Required",           # Scheduling for this week, stock inquiries
        "48-Hour Response Needed",      # Quote needed within 2 days, upcoming appointment
        "Weekend Preparation",          # Planning for weekend events/services
        
        # Standard Timing
        "Regular Booking",              # Normal appointment scheduling
        "Standard Order Process",       # Typical product orders/inquiries
        "Routine Inquiry",             # General questions about services/products
        
        # Future Planning
        "Advance Booking",             # Events, large projects, seasonal orders
        "Project Planning",            # Custom work, renovations, design projects
        "Seasonal Planning",           # Holiday orders, seasonal services
        
        # Follow-up Based
        "Post-Service Check",          # After service completion
        "Maintenance Schedule",        # Regular service planning
        "Project Update"               # Progress updates on ongoing work
    ]
)

In [11]:
data_designer.add_categorical_seed_column(
    name="business_relationship",
    description="The nature of the customer-business relationship in a small business context",
    values=[
        # New Relationships
        "First-Time Local Customer",             # Local resident trying service for first time
        "Word-of-Mouth Referral",               # Referred by existing customer
        "Social Media Discovery",               # Found business through local social media
        "Local Event Introduction",             # Met at farmer's market/community event
        
        # Established Relationships
        "Weekly Regular",                       # E.g., Standing yoga class, weekly bread order
        "Monthly Service Client",               # Regular maintenance, monthly appointments
        "Seasonal Customer",                    # Holiday orders, seasonal services
        "Project-Based Client",                 # Ongoing renovation, wedding planning
        
        # Community Connections
        "Fellow Local Business Owner",          # Other business owner in community
        "Local Family Customer",                # Family with multiple service needs
        "Neighborhood Regular",                 # Lives/works nearby, frequent casual visits
        "Community Event Partner",              # Collaborated on local events
        
        # Special Circumstances
        "Multi-Generation Customer",            # Family has used business for years
        "Former Regular Returning",             # Coming back after moving/break
        "Special Needs Client",                 # Requires specific accommodations
        "Custom Order Regular"                  # Regular custom/specialized orders
    ]
)

In [12]:

data_designer.add_categorical_seed_column(
    name="email_length",
    description="Number of email exchanges in the conversation",
    values=[2, 3, 4, 5, 6]
)

### ✨ Generated Data Columns
We use three main generated columns to create our email conversations:

#### Email Objective

* Creates specific, detailed scenarios for email communication
* Ensures the objective matches the business type and conversation goal
* Maintains realism and relevance

#### Email Subject

* Generates appropriate subject lines based on context
* Reflects urgency and tone
* Matches natural email writing patterns

#### Email Contents

* Generates the full email exchange
* Maintains consistent personality traits
* Includes appropriate business details
* Reaches natural conclusions

Each generated column builds upon the previous ones to create coherent, realistic email conversations that feel authentic to small business communications.

In [13]:
data_designer.add_generated_data_column(
    name="business_name",
    generation_prompt=(
        "Generate a name for a small business. The business type is: {business_type}, specifically: {specific_business}"
    )
)

In [14]:
data_designer.add_generated_data_column(
    name="email_objective",
    generation_prompt=(
        "Based on this small business context:\n"
        "- Business Type: {business_type} (Specifically: {specific_business})\n"
        "- Conversation Goal: {conversation_goal}\n"
        "- Customer Type: {business_relationship}\n"
        "\n"
        "Generate ONE clear, specific reason why a customer needs to email this small business. "
        "The objective should:\n"
        "- Focus on a single, coherent need or request\n"
        "- Include relevant details that matter to the business owner\n"
        "- Be realistic for this specific type of business\n"
        "- Match the conversation goal\n"
        "\n"
        "Examples:\n"
        "- For a bakery: 'Need to order a gluten-free birthday cake for 15 people, decorated with a garden theme, required for next Saturday afternoon.'\n"
        "- For a plumber: 'Kitchen sink is backing up and making gurgling sounds. Need assessment and repair quote, preferably within next two days.'\n"
        "- For a yoga studio: 'Interested in private yoga sessions for a group of 4 beginners, looking to schedule weekly classes starting next month.'\n"
        "\n"
        "Only provide the objective, no additional context or explanation. Keep it focused on one clear purpose."
    )
)

In [15]:

# Generate an appropriate subject line
data_designer.add_generated_data_column(
    name="email_subject",
    generation_prompt=(
        "Create an email subject line based on:\n"
        "- Email Objective: {email_objective}\n"
        "- Urgency Level: {urgency_level}\n"
        "- Conversation Tone: {conversation_tone}\n"
        "\n"
        "The subject line should:\n"
        "- Be one line only\n"
        "- Reflect the urgency when needed\n"
        "- Be specific enough to be useful\n"
        "- Match how real people write email subjects to small businesses\n"
        "- Sound natural, not corporate\n"
        "\n"
        "Only provide the subject line, no quotes or additional context."
    )
)

In [16]:

# Generate the email conversation
data_designer.add_generated_data_column(
    name="email_contents",
    generation_prompt=(
        "Generate an email conversation between a small business owner and customer with these parameters:\n"
        "\n"
        "CONTEXT:\n"
        "- Business Type: {business_type} (Specifically: {specific_business})\n"
        "- Owner's Style: {owner_personality}\n"
        "- Customer's Style: {customer_personality}\n"
        "- Relationship: {business_relationship}\n"
        "- Customer's Tone: {conversation_tone}\n"
        "- Business Name: {business_name}\n"
        "\n"
        "OBJECTIVE:\n"
        "{email_objective}\n"
        "\n"
        "SUBJECT LINE:\n"
        "{email_subject}\n"
        "\n"
        "REQUIREMENTS:\n"
        "- The business owner's tone should be a natural response to the customer's tone and the objective of the email.\n"
        "- The conversation must have {email_length} number of emails and reach a conclusion by the end of it.\n"
        "- Start with the customer's initial email\n"
        "- Each email should feel like a real small business interaction\n"
        "- Make sure you reflect the communication styles of both owner and customer throughout the email chain\n"
        "- Include natural details and follow-ups\n"
        "- End when the objective is achieved or an appropriate next step is established\n"
        "- Use realistic email formatting\n"
        "- Keep the tone authentic to a small business interaction\n"
        "- Give the business owner and customer realistic names, sign each email with the person's name.\n"
        "- Make sure the names used are consistent across the entire email chain."
    ),
    data_config={"type": "structured", "params": {"model": EmailConversation}},
    llm_type="judge"
)

### 👀 Generating and Reviewing Email Previews
Preview mode lets you quickly validate and refine your email generation setup. Each preview run creates 10 sample email conversations.

In [17]:
preview = data_designer.generate_dataset_preview(verbose_logging=True)

[19:37:41] [INFO] 🚀 Generating dataset preview
[19:37:42] [INFO] 🦜 Step 1: Generate seed category values
[19:37:42] [INFO]   |   |-- ✨ Generating values for seed subcategory `specific_business` when `business_type` is Food & Beverage
[19:37:42] [INFO]   |   |-- ✨ Generating values for seed subcategory `specific_business` when `business_type` is Local Retail
[19:37:43] [INFO]   |   |-- ✨ Generating values for seed subcategory `specific_business` when `business_type` is Skilled Trade
[19:37:43] [INFO]   |   |-- ✨ Generating values for seed subcategory `specific_business` when `business_type` is Creative Services
[19:37:43] [INFO]   |   |-- ✨ Generating values for seed subcategory `specific_business` when `business_type` is Health & Wellness
[19:37:44] [INFO]   |-- Model usage: [{"model": "Qwen/Qwen2.5-7B-Instruct", "prompt_tokens": 2261, "completion_tokens": 90, "request_count": 5, "total_tokens": 2351}]
[19:37:44] [INFO] 🎲 Step 2: Sample data seeds
[19:37:44] [INFO]   |-- 🎲 Randomly sam

### 🔎 Inspecting Individual Conversations

Run the display cell below to examine individual email conversations. Each run shows you a different preview record.

In [21]:
preview.display_sample_record()

                                             Categorical Seed Columns                                              
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Name                                                ┃ Value                                                     ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ customer_personality                                │ Confused and Seeking Help                                 │
├─────────────────────────────────────────────────────┼───────────────────────────────────────────────────────────┤
│ urgency_level                                       │ Seasonal Planning                                         │
├─────────────────────────────────────────────────────┼───────────────────────────────────────────────────────────┤
│ owner_personality                                   │ Creative and Casual                                       │
├─────────────────────────────────────────────────────┼───────────────────────────────────────────────────────────┤
│ conversation_tone                                   │ Friendly                                                  │
├─────────────────────────────────────────────────────┼───────────────────────────────────────────────────────────┤
│ business_relationship                               │ Word-of-Mouth Referral                                    │
├─────────────────────────────────────────────────────┼───────────────────────────────────────────────────────────┤
│ conversation_goal                                   │ Custom Project Proposal                                   │
├─────────────────────────────────────────────────────┼───────────────────────────────────────────────────────────┤
│ email_length                                        │ 2                                                         │
├─────────────────────────────────────────────────────┼───────────────────────────────────────────────────────────┤
│ business_type                                       │ Health & Wellness                                         │
├─────────────────────────────────────────────────────┼───────────────────────────────────────────────────────────┤
│   |- specific_business                              │ Local Yoga Studio                                         │
└─────────────────────────────────────────────────────┴───────────────────────────────────────────────────────────┘
                                                                                                                   
                                                                                                                   
                                              Generated Data Columns                                               
┏━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Name            ┃ Value                                                                                         ┃
┡━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ business_name   │ How about "Sunrise Yoga Studio"? It sounds inviting and reflects a local, community-focused   │
│                 │ approach to yoga and wellness.                                                                │
├─────────────────┼───────────────────────────────────────────────────────────────────────────────────────────────┤
│ email_objective │ Interested in exploring potential sponsorship opportunities for your studio at our upcoming   │
│                 │ local health fair next month. Looking to discuss details and see if it aligns with your       │
│                 │ goals.                                                                                        │
├─────────────────┼─────────────────────────────────────

### 🤔 Like what you see?

Submit a batch workflow!

In [19]:
# Submit batch job
batch_job = data_designer.submit_batch_workflow(num_records=100)
df = batch_job.fetch_dataset(wait_for_completion=True)
print("\nGenerated dataset shape:", df.shape)

By following these steps and leveraging the interactivity of the SDK, you can refine prompts, generate realistic dialogues, and ensure the resulting dataset is high-quality, non-toxic, and aligned with your domain-specific requirements.

In [20]:
# Inspect first 10 records of the generated dataset
df.head(10)